In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import csv
from datetime import datetime

/Users/lucia/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


* Mood vector Design 1: for each day, if negative_count > positive_count or negative_count > neutral_count or neative_count > mix_count, assign negative to that day; if positive_count > negative_count or positive_count > neutral_count or positive_count > mix_count, assign positive to that day; if mix_count >= positive_count or mix_count >= negative_count or negative_count == positive_count, assign mix to that day; if neutral_count >= positive_count or neutral_count >= negative_count or neutral_count > mix_count, assign neutral to that day.

In [3]:
time = pd.read_csv('../../data/important_data/twoM_newLabels.csv')
ids = pd.read_csv('../../data/important_data/Id80PerRetained.csv')
time  = time[['text','id','negative_ny','userid','time','time_diff']]
time.shape

(4099, 6)

In [4]:
ids2 = ids[['userid']]
time = pd.merge(ids2, time, how = 'left', on = 'userid' )
time.shape

(4086, 6)

In [5]:
#select date
time['time'] = time['time'].apply(lambda x: x.split()[0])
time[1:5]

,userid,text,id,negative_ny,time,time_diff
1,009d96a823b6f6c085c092fb177491f6,"In honor of Mother's Day, we are trying to see...",948,4,05/05/2011,7
2,009d96a823b6f6c085c092fb177491f6,"What would your liqour do? If it could, it wou...",949,2,05/05/2011,7
3,009d96a823b6f6c085c092fb177491f6,War is about who's left not who's right.... be...,946,2,04/05/2011,8
4,009d96a823b6f6c085c092fb177491f6,Loving the fact my babie made me breakfast mmm...,942,2,01/05/2011,11


In [6]:
#convert to time series
# time['time'] = time['time'].apply(lambda x: parser.parse(x))
# time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
time['time'][1:10]

1   2011-05-05
2   2011-05-05
3   2011-05-04
4   2011-05-01
5   2011-04-28
6   2011-04-25
7   2011-04-20
8   2011-04-20
9   2011-04-19
Name: time, dtype: datetime64[ns]

In [8]:
#sort values according to userid, time and valence
time = time.sort_values(by=['userid','time_diff','negative_ny'],  ascending=False)
time[0:5]

,userid,text,id,negative_ny,time,time_diff
4085,fbd681e79ebca2cd9509218403f23ec6,"Mirfie's coming home tomorrow!! TOORAY, TOORAY!",4103,2,2010-12-18,59
4083,fbd681e79ebca2cd9509218403f23ec6,Happy birthday Giancarlo!!!!!!!,4105,2,2010-12-20,57
4084,fbd681e79ebca2cd9509218403f23ec6,"I want a Christmas tree, I want a Christmas tr...",4106,2,2010-12-20,57
4082,fbd681e79ebca2cd9509218403f23ec6,SHOOTAH!,4107,2,2010-12-22,55
4081,fbd681e79ebca2cd9509218403f23ec6,"(oh, you're making me live) ... Oh, you're the...",276,2,2010-12-23,54


In [18]:
removed = time.groupby('userid').head(1).reset_index()
len(removed[removed['time_diff'] != 60])

40

In [10]:
#count_post = time.groupby(['userid']).size().reset_index(name='counts')
# #remove users with less than 10 post, they are acutally users who have 80% of their foreign language posts removed 
#clean = count_post.loc[count_post['counts'] >= 16, ]
#clean.shape

Create user vector

In [11]:
def user_obj(dfUserid):
    users = {}
    for user in dfUserid:
       # print(user)
        if user not in users:
            users[user] = [-1]*60 
    return users

#get the which valence is dominate in one day
def getValenceFromMultipleDays(curdayPosts):
    positive_count = 0
    neg_count = 0
    neu_count = 0
    mix_count = 0
    for post in curdayPosts:
        if post == 1:
            neg_count += 1
        elif post == 3:
            mix_count += 1
        elif post == 4:
            neu_count += 1
        else:
            positive_count +=1
#     print(mix_count < positive_count or mix_count < neg_count)
#     print('neg_count{}, pos_count{}, neu_count{}, mix_count{}'. format(neg_count,positive_count,neu_count,mix_count))
    if (neg_count !=0 and (neg_count > positive_count or neg_count > neu_count or neg_count > mix_count )):
        return 1
    #give more weights to mix
    if (mix_count !=0 and (mix_count >= positive_count or mix_count >= neg_count or neg_count == positive_count)):
        return 3
    #here we give positive emotions more weights
    if (positive_count !=0 and (positive_count > neg_count or positive_count > neu_count or positive_count > mix_count)):
        return 2
    elif (neu_count !=0 and (neu_count >= positive_count or neu_count >= neg_count or neu_count > mix_count)):
        return 4
    else:
        return -1

def getValenceVector(userObject, df):
    preDay = None
    preUser = None
    preValence = None
    curdayPosts = []
    i = 0
    for valence, day, user in zip(df['negative_ny'], df['time_diff'], df['userid']):
        #rint(user, day)
        #posVal = 0
        if preUser is None:
            curdayPosts = [valence]
        elif day == preDay and user == preUser:# and valence != preValence:
            curdayPosts.append(valence)
        else:
            dayvalence = getValenceFromMultipleDays(curdayPosts)
            curdayPosts = [valence]
            userObject[user][int(day)-1] = dayvalence
        preDay = day
        preUser = user
        i +=1
    return userObject

In [12]:
users = user_obj(time['userid'])
users2 = getValenceVector(users, time)

In [13]:
count = 0
emptyD = []
negativeD = []
positiveD = []
neutralD = []
mixed = []
for userid in users2:
    emptyD.append(users[userid].count(-1))
    negativeD.append(users[userid].count(1))
    positiveD.append(users[userid].count(2))
    neutralD.append(users[userid].count(4))
    mixed.append(users[userid].count(3))
    print('user {} has {} empty days, {} negative days, {} positive days {}, neutral days and mix days {}'.format(userid, users[userid].count(-1), users[userid].count(1), users[userid].count(2), users[userid].count(4), users[userid].count(3)))
    count += 1
count


user fbd681e79ebca2cd9509218403f23ec6 has 31 empty days, 10 negative days, 17 positive days 2, neutral days and mix days 0
user f8708568eff1535c85247b28dc126f9b has 1 empty days, 32 negative days, 23 positive days 2, neutral days and mix days 2
user f49956142aab2ad74decfd49a103c2f1 has 29 empty days, 11 negative days, 13 positive days 7, neutral days and mix days 0
user f358b75c71d85c91ec0452c6114e3b15 has 28 empty days, 7 negative days, 20 positive days 3, neutral days and mix days 2
user ec3dc451b8a811f05ace158a0d76e32e has 43 empty days, 9 negative days, 2 positive days 6, neutral days and mix days 0
user eaeabcfe25866f0455d349bae2450d88 has 27 empty days, 11 negative days, 18 positive days 3, neutral days and mix days 1
user e9970fe8848e9505b52d414d81611de8 has 40 empty days, 5 negative days, 10 positive days 3, neutral days and mix days 2
user e076953cd3fcf8927b8f21574c0890f3 has 27 empty days, 9 negative days, 16 positive days 7, neutral days and mix days 1
user dcbe08425f5edef5e

74

statistics

In [14]:
df = pd.DataFrame(np.array(emptyD).reshape(74,1), columns=['EmptyDays'])
df['NegativeDays'] = negativeD
df['PositiveDays'] = positiveD
df['NeutralDays'] = neutralD
df['MixedDays'] = mixed
df[1:5]

,EmptyDays,NegativeDays,PositiveDays,NeutralDays,MixedDays
1,1,32,23,2,2
2,29,11,13,7,0
3,28,7,20,3,2
4,43,9,2,6,0


In [12]:
df.describe()

,EmptyDays,NegativeDays,PositiveDays,NeutralDays,MixedDays
count,74.000000,74.000000,74.000000,74.000000,74.000000
mean,28.364865,12.756757,14.054054,4.000000,0.824324
std,11.422849,9.267428,6.702876,2.923687,0.998054
min,0.000000,0.000000,2.000000,0.000000,0.000000
25%,23.250000,7.000000,10.000000,2.000000,0.000000
50%,30.000000,10.000000,14.000000,3.000000,1.000000
75%,37.000000,15.000000,17.000000,6.000000,1.000000
max,43.000000,54.000000,46.000000,14.000000,5.000000


In [16]:
#pickle
import pickle
with open('../../data/important_data/moodVectorsData/MoodVecDes1.pickle', 'wb') as handle:
    pickle.dump(users2, handle, protocol=pickle.HIGHEST_PROTOCOL)

Now we write the valence vector as csv file 

In [17]:
#write result as csv
#header = ['userid','vector']
with open('../../data/important_data/moodVectorsData/ValenceVec_Norm_Clean.csv','w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(i for i in header)
    writer.writerow(users.keys())
    for row in zip(*users.values()):
        writer.writerow(list(row))

In [18]:
valenceVec = pd.read_csv('../../data/important_data/moodVectorsData/ValenceVec_Norm_Clean.csv')
valenceVec[1:5]

,fbd681e79ebca2cd9509218403f23ec6,f8708568eff1535c85247b28dc126f9b,f49956142aab2ad74decfd49a103c2f1,f358b75c71d85c91ec0452c6114e3b15,ec3dc451b8a811f05ace158a0d76e32e,eaeabcfe25866f0455d349bae2450d88,e9970fe8848e9505b52d414d81611de8,e076953cd3fcf8927b8f21574c0890f3,dcbe08425f5edef5e5c1053f21e83640,db7f1b0130b2138e13b7bbca9cc63823,...,1e84bfe1c9bbd549913aad1bc57c8878,1c1a61d11a10f9fa967f250095792db3,193165601dfad67e79ef52b2caf9fd9e,1789589a6d2bba0f09d768f8f3445be5,0e3fdb89c7706b13891170ab2a9017c7,0c5d06511ef79fba34da417e8c751fff,0b12516100c16f779b152858619786ff,088bf0ed841bcc366475b3975eb1b9d2,0724fe854bd455061ba84efecdeff469,009d96a823b6f6c085c092fb177491f6
1,-1,1,-1,2,4,2,-1,1,-1,-1,...,-1,2,-1,2,1,-1,1,2,-1,-1
2,-1,1,-1,2,1,-1,-1,-1,-1,2,...,2,1,-1,-1,-1,-1,1,-1,-1,-1
3,-1,1,2,1,4,2,-1,1,2,-1,...,-1,-1,-1,-1,1,-1,-1,-1,3,2
4,-1,1,1,-1,2,1,-1,2,1,-1,...,1,2,2,1,3,-1,2,-1,-1,-1


In [19]:
valenceVec = valenceVec.transpose()
valenceVec[1:5]

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
f8708568eff1535c85247b28dc126f9b,1,1,1,1,1,1,3,2,1,-1,...,1,2,2,1,1,1,2,1,2,2
f49956142aab2ad74decfd49a103c2f1,-1,-1,-1,2,1,4,2,2,-1,2,...,-1,-1,1,4,4,-1,1,1,2,-1
f358b75c71d85c91ec0452c6114e3b15,-1,2,2,1,-1,2,-1,2,-1,4,...,-1,2,2,2,-1,2,-1,-1,-1,-1
ec3dc451b8a811f05ace158a0d76e32e,1,4,1,4,2,-1,4,1,1,-1,...,-1,-1,-1,-1,-1,2,-1,-1,-1,-1


need to mannually change first column to userid

In [20]:
valenceVec.to_csv('../../data/important_data/moodVectorsData/ValenceVec_Norm_Clean.csv')